OSCAR can also do polyhedral geometry! All of the polymake functionality is accessible through OSCAR. 

In [ ]:
using Oscar 

## Polytopes 
Let's load some "standard" polytopes 


In [ ]:
standard_simplex = simplex(3)

In [ ]:
two_cube = cube(2) 

In [ ]:
is_simplicial(standard_simplex), is_simple(two_cube), is_simplicial(two_cube), is_simplicial(cube(3)) 

In [ ]:
[1,1,1] in standard_simplex, [0,0,0] in standard_simplex, interior_lattice_points(two_cube) 

print_constraints allows us to pass between V and H-descriptions: 

In [ ]:
print_constraints(two_cube) 

In [ ]:
vertices(polarize(two_cube))

In [ ]:
2*two_cube 


In [ ]:
two_cube + standard_simplex #throws an error. Polytopes should be of the same dimension! 

In [ ]:
is_lattice_polytope(polarize(2*two_cube)) 

## Cones

In [ ]:
C1 = positive_hull([ 1 2 ; 0 1]) 
#dim(C1) 
#rays(C1) 
#hilbert_basis(C1)

In [ ]:
C2 = cone_from_inequalities([1 0 0; 0 1 0; 1 1 0])
#is_pointed(C2)
#lineality_space(C2) 
#[0,0,1] in C2
#hilbert_basis(C2) 

In [ ]:
rays(polarize(C2)) 
is_pointed(polarize(C2)) 
#intersect(C1, C2) #throws an error. Why? 
#visualize 

normal_fan(two_cube) 
#rays(ans) 

## Gröbner Cones
Consider the ideal $$I = \langle x^2 + yz \ , \  xy + z^2 \rangle \triangleleft \mathbb{Q}[x,y,z]$$

Let's compute its degree reverse lexicographic Gröbner basis: 

In [ ]:
R, (x,y,z) = polynomial_ring(QQ, ["x", "y", "z"]) #define the ring

I = ideal([x^2 + y*z, x*y + z^2]) #define the ideal
 
G = groebner_basis(I) #computes a degree reverse lexicographic Gröbner basis

Recall that the Gröbner cone of a Gröbner basis $G_<$ is the euclidean closure of the set of weight vectors which represent $G$ w.r.t $<$:     \begin{equation}
    C_{<} = \overline{\{ \omega \in \mathbb{R}^n \text{ : } in_<(g_i) = in_\omega(g_i) \text{  for  } i = 1,...s \} } \cap GR(I)
\end{equation}

In [ ]:

function bounding_vectors(G::Oscar.IdealGens)
  gens_by_terms = terms.(G; ordering=ordering(G))
  
  v = map(Iterators.peel.(gens_by_terms)) do (lead,tail)
      Ref(leading_exponent_vector(lead)) .- leading_exponent_vector.(tail)
  end

  return unique!(reduce(vcat, v))
end
bounding_vectors(G) #these vectors give a hyperplane description of the Gröbner cone

#gens_by_terms = terms.(G; ordering=ordering(G))
#= 
  v = map(Iterators.peel.(gens_by_terms)) do (lead,tail)
      Ref(leading_exponent_vector(lead)) .- leading_exponent_vector.(tail) 
end 
=# 



In [ ]:
C = cone_from_inequalities(-bounding_vectors(G))
conerays = rays_modulo_lineality(C)[1]
w = sum(conerays) 
#w =relative_interior_point(C)

all([dot(w, v) > 0 for v in bounding_vectors(G)]) #w 
w = ZZ.(collect((inv(gcd(w)))*w))

In [ ]:
o2 = weight_ordering(w, degrevlex(R))
G2 = groebner_basis(I, ordering = o2)
G == G2 
gens(G) == gens(G2) 
Set(gens(G)) == Set(gens(G2))

## Applications of Gröbner fan #1: The Gröbner Walk 
The theory of Gröbner fans can be used to speed up Gröbner bases computation!
(see slides) 
